# Основы АД в МО. Введение в Python: модуль pandas

## Работа с модулем Pandas
Напомним, что такое модуль: в модуле хранятся функции, которые "не вместились" в основной синтаксис языка. Например, рассматривался модуль `math`, который содержал функции из продвинутого калькулятора.

![](https://filearmy.s3.amazonaws.com/2017/03/03/25b10be.jpg)

Модуль Pandas предназначен для работы с данными, чем-то напоминающей работу в Excel. Данные хранятся в таблице с именованными колонками и пронумерованными строками.

Для начала подключим модуль:

In [ ]:
# Данная конструкция подключает модуль pandas
# и позволяет обращаться к его функциям через имя pd.
import pandas as pd

### Чтение таблицы из файла
В `pandas` можно легко и удобно считывать таблицы форматов .csv, .tsv, .xls, .xlsx, ... с помощью функции `read_csv` или `read_excel` в зависимости от формата. В течение курса мы в основном будем считывать из .csv (comma separated values).

Рассмотрим таблицу, содержащую различные показатели по странам с $1980$ по $2009$ год, взятые из базы [United Nations System](http://data.un.org/Default.aspx).

Считаем данные и выведем первые $5$ строк:

In [ ]:
countries = pd.read_csv('country_statistics.csv')
countries.head(5)

Можно узнать размер таблицы (количество строк и столбцов):

In [ ]:
countries.shape

Можно вывести названия всех столбцов:

In [ ]:
countries.columns

Краткое описание таблицы можно вывести так:

In [ ]:
countries.info()

Из полезного: можно сразу же оценить, сколько пропусков в данных.

### Простейшие операции

Из таблицы можно получать отдельные колонки, обращаясь к ним по именам, например, так:

In [ ]:
countries.Year

Однако такой способ не сработает, если колонка содержит пробелы (в нашем случае таких большинство). Поэтому в `pandas` предусмотрено эквивалентное решение:

In [ ]:
countries['Year']

In [ ]:
countries['Country or Area']

Также, можно выводить сразу несколько колонок. Для этого надо передать **список** названий колонок (т.е. после имени таблицы появятся двойные квадратные скобки `[]`):

In [ ]:
countries[['Country or Area', 'Year']]

Вывести строки с конкретным значением в столбце можно так:

In [ ]:
# вывод строк с информацией за 1995 год
countries[countries.Year == 1995]

### Комбинация условий
Иногда бывает нужно вывести строки с фиксированными значениями нескольких полей. В таких случаях можно воспользоваться логикой, аналогичной той, что использовалась в конструкции `if`.

Для проверки нескольких условий в `if` мы писали конструкции вроде такой:

In [ ]:
if 1 > 0 and 10 <= 11:
    print('ok')

Аналогичную конструкцию можно вписать в квадратных скобках `[]` для умной индексации в `pandas`:

In [ ]:
# вывод строк с информацией за 1995 год и средней продолжительностью жизни больше 77
countries[(countries['Year'] == 1995) & (countries['Life expectancy'] > 77)]

Важные отличия от `if`:
- оператор `and` заменяется на символ `&`
- оператор `or` заменяется на символ `|`
- оператор `not` заменяется на символ `~`
- каждое условие надо оборачивать в круглые скобки `()`

____
#### Задание
Выведите информацию за 2000 год, где ВВП меньше 500.

In [ ]:
#YOUR CODE

### Работа с пропусками
Как видно по таблице, при сборе данных неизбежно возникают *пропущенные значения* и в таблице появляются поля со значением NaN (Not a Number).

Есть разные способы от них избавиться:
- можно заменить их на какое-нибудь фиксированное значение
- можно вообще удалить строки, содержащие NaN
- в некоторых задачах NaN не мешают вовсе

На количество пропущенных значений можно посмотреть в сводной статистике:

In [ ]:
countries.info()

Одна из мер борьбы с NaN -- замена на какое-либо значение. Выведем самое частое значение (моду) показателя `Life expectancy`:

In [ ]:
countries['Life expectancy'].value_counts(sort=True)

Заменим пропущенные значения в столбце `Life expectancy` на самое частое значение этого столбца: $71.5$.

In [ ]:
# Копируем исходную таблицу, чтобы не портить данные.
countries_without_nan = countries.copy()

In [ ]:
countries_without_nan['Life expectancy'] = countries_without_nan['Life expectancy'].fillna(71.5)

Важно избегать необдуманных замен значений в данных наподобие совершённого. Такие действия могут существенно повлиять на описательные статистики, так что иногда безопаснее по возможности просто не рассматривать данные с пропусками:

In [ ]:
countries_without_nan = countries_without_nan.dropna(how='any')

Значение аргумента `how='any'` означает, что будут удалены все строки, содержащие **хотя бы один** пропуск.

Можно было прописать `how='all'`, в результате чего удалились бы только строки, полностью состоящие из NaN.

In [ ]:
countries_without_nan.info()

Опять же, важно понимать, что вы можете потерять очень много информации:

In [ ]:
countries.shape, countries_without_nan.shape

### Описательные статистики
У таблицы в `pandas` есть множество методов для получения тех или иных описательных статистик:

In [ ]:
print('Mean life expectancy: ', countries['Life expectancy'].mean())
print('Min life expectancy: ', countries['Life expectancy'].min())
print('Life expectancy 25% quantile: ', countries['Life expectancy'].quantile(0.25))

Для сравнения, те же значения на таблице с неаккуратно вычищенными пропусками:

In [ ]:
print('Mean life expectancy: ', countries_without_nan['Life expectancy'].mean())
print('Min life expectancy: ', countries_without_nan['Life expectancy'].min())
print('Life expectancy 25% quantile: ', countries_without_nan['Life expectancy'].quantile(0.25))

Также, можно выводить целые наборы статистик по числовым полям таблицы:

In [ ]:
countries.describe()

- count - число наблюдений без пропусков
- mean - среднее значение
- std - стандартное отклонение
- min - минимум
- 50% - медиана
- 25% - 25% квантиль
- 75% - 75% квантиль
- max - максимум

Можно построить такую же таблицу для категориальных переменных:

In [ ]:
countries.describe(include='object')

- count - число наблюдений без пропусков
- unique - число уникальных значений
- top - самое частое значение
- freq - частота, с которой встречается значение top

### Задание: fun with pandas
Напомним, как выглядит табличка с данными по странам:

In [ ]:
countries.head()

Выведите следующие значения:
1. Максимальный процент детей с избыточным весом.
2. Максимальный процент детей с недостатком веса в $2000$-$2005$ гг..
3. Страны с продолжительностью жизни, превышающей $75\%$ квантиль в промежуток с $1990$ по $2000$ год.
4. Страны, в которых было больше детей с недостатком веса, чем детей с избытком, с $2000$ года.
5. Среднее значение ВВП с $1995$ по $2000$ год.

In [ ]:
# YOUR CODE

In [ ]:
# YOUR CODE

In [ ]:
# YOUR CODE

In [ ]:
# YOUR CODE

In [ ]:
# YOUR CODE

### Дополнительная обработка
Иногда, когда мы скачиваем данные из источников, бывает, что они выглядят не так, как мы предполагали. Например, числовые значения записаны как строки (показатель Literacy). Для этого требуется дополнительная обработка.

In [ ]:
countries_without_nan.dtypes

In [ ]:
countries_without_nan['Literacy']

Показатели в столбце Literacy должны быть числами (float), а не строками. Для того, чтобы преобразовать, уберем знак `%` из каждого значения и переведем в число. Можем обрабатывать таким образом только столбцы, где уже нет NaN.

In [ ]:
countries_without_nan['Literacy'] = countries_without_nan['Literacy'].apply(lambda x: x.replace('%', ''))
countries_without_nan['Literacy'] = countries_without_nan['Literacy'].apply(float)

In [ ]:
countries_without_nan.dtypes